# Data Cleaning und Handling


### Imports

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import os
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
import numpy as np
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## Code

### Dowloading Data

In [3]:
#response = requests.get("https://github.com/MarcusGrum/AIBAS/blob/main")
#response = requests.get("https://www.kaggle.com/datasets/victormegir/bikes-from-bikezcom/data")

#doc = BeautifulSoup(response.text, "html.parser")

#article = doc.find("article")

#print(doc)
#print("testpush")
#table = article.find("table")

# df = pd.read_html(str(table))[0].copy()
# print(df)


"""
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "bikes.csv"

# Load the latest version
df = kagglehub.dataset_load(
  KaggleDatasetAdapter.PANDAS,
  "victormegir/bikes-from-bikezcom",
  file_path,
  # Provide any additional arguments like 
  # sql_query or pandas_kwargs. See the 
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

import kagglehub

# Download latest version
path = kagglehub.dataset_download("victormegir/bikes-from-bikezcom")

print("Path to dataset files:", path)
"""

'\nimport kagglehub\nfrom kagglehub import KaggleDatasetAdapter\n\n# Set the path to the file you\'d like to load\nfile_path = "bikes.csv"\n\n# Load the latest version\ndf = kagglehub.dataset_load(\n  KaggleDatasetAdapter.PANDAS,\n  "victormegir/bikes-from-bikezcom",\n  file_path,\n  # Provide any additional arguments like \n  # sql_query or pandas_kwargs. See the \n  # documenation for more information:\n  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas\n)\n\nimport kagglehub\n\n# Download latest version\npath = kagglehub.dataset_download("victormegir/bikes-from-bikezcom")\n\nprint("Path to dataset files:", path)\n'

### Cleaning Up the Data

### Drop NAN

Da bei Drop NAN für

In [4]:
current=current = os.getcwd()
parent=parent_dir = os.path.dirname(current)

df=pd.read_csv(parent_dir+"/datasets/victormegir/bikes-from-bikezcom/versions/4/bikes.csv")

##---------------Sichere Auswahl mit DropNA---------------------##

kat_selection_safe=["Category","Engine type","Transmission type,final drive","Displacement ccm","Fuel capacity liters","Power HP","Front brakes","Rear brakes"]
kat_selection_all=["Category","Engine type","Transmission type,final drive","Displacement ccm","Fuel capacity liters","Power HP","Front brakes","Rear brakes","Starter","Seat height mm","Dry weight kg","Overall length mm","Wheelbase mm","Top speed km/h"]

df_selected_safe=df[kat_selection_safe].copy()
df_selected_all=df[kat_selection_all].copy()

#print(df_all.shape)

df_clean=df_selected_safe.dropna().copy()
df_all_0=df_selected_all.fillna(0).copy()

print(df_all_0.head())

       Category                   Engine type Transmission type,final drive  \
0         Sport  Single cylinder, four-stroke                         Chain   
1  Super motard               V2, four-stroke                         Chain   
2         Sport  Single cylinder, four-stroke                         Chain   
3         Sport  Single cylinder, four-stroke                         Chain   
4         Sport   Single cylinder, two-stroke                         Chain   

   Displacement ccm  Fuel capacity liters  Power HP  \
0             600.0                  17.0       0.0   
1             896.1                  12.0      95.2   
2             124.2                  14.5      28.2   
3             124.2                  14.5      15.0   
4              50.0                  14.5       0.0   

                                        Front brakes  \
0              Single disc. Brembo 2-piston calipers   
1  Double disc. ABS.Stainless steel wave floating...   
2  Single disc. ABS. Stain

C:\Users\david\AppData\Local\Temp\ipykernel_33008\751862266.py:4: DtypeWarning: Columns (38) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(parent_dir+"/datasets/victormegir/bikes-from-bikezcom/versions/4/bikes.csv")


### Editing values

In [5]:
##-------Bis zum Punkt steht doppel/Single etc-------------##
df_clean["Front brakes"]=df_clean['Front brakes'].str.split('.').str[0]
df_clean["Rear brakes"]=df_clean['Rear brakes'].str.split('.').str[0]
print(df_clean.shape)
# mindest Wert denn eine Art haben muss an Datensatzhäufigkeit
min_count = 200

# Häufigkeiten zählen
value_counts = df_clean['Category'].value_counts()
#print(value_counts)

# Kategorien mit genug Vorkommen identifizieren
valid_categories = value_counts[value_counts >= min_count].index
print(valid_categories)

# DataFrame filtern
df_clean = df_clean[df_clean['Category'].isin(valid_categories)].copy()
print(df_clean.shape)
df_clean


(18847, 8)
Index(['Sport', 'Scooter', 'Naked bike', 'Allround', 'Custom / cruiser',
       'Classic', 'Enduro / offroad', 'Sport touring', 'Touring',
       'Super motard', 'Unspecified category', 'ATV', 'Cross / motocross'],
      dtype='object', name='Category')
(18573, 8)


,Category,Engine type,"Transmission type,final drive",Displacement ccm,Fuel capacity liters,Power HP,Front brakes,Rear brakes
1,Super motard,"V2, four-stroke",Chain,896.1,12.0,95.2,Double disc,Single disc
2,Sport,"Single cylinder, four-stroke",Chain,124.2,14.5,28.2,Single disc,Single disc
3,Sport,"Single cylinder, four-stroke",Chain,124.2,14.5,15.0,Single disc,Single disc
5,Sport,"Twin, four-stroke",Chain,659.0,15.0,100.0,Double disc,Single disc
6,Sport,"Twin, four-stroke",Chain,659.0,15.0,46.9,Double disc,Single disc
...,...,...,...,...,...,...,...,...
38240,Naked bike,"Two cylinder boxer, four-stroke",Shaft drive (cardan),494.0,14.0,8.6,Expanding brake (drum brake),Expanding brake (drum brake)
38272,Naked bike,"Two cylinder boxer, four-stroke",Shaft drive (cardan),494.0,14.0,8.6,Expanding brake (drum brake),Expanding brake (drum brake)
38305,Naked bike,"Two cylinder boxer, four-stroke",Shaft drive (cardan),494.0,14.0,8.6,Expanding brake (drum brake),Expanding brake (drum brake)
38339,Naked bike,"Two cylinder boxer, four-stroke",Shaft drive (cardan),494.0,14.0,8.6,Expanding brake (drum brake),Expanding brake (drum brake)


In [6]:
numeric_columns = df_clean.select_dtypes(include=['number']).columns.tolist()

def removeOutliers(data, col):
    Q3 = np.quantile(data[col], 0.75)
    Q1 = np.quantile(data[col], 0.25)
    IQR = Q3 - Q1

    print("IQR value for column %s is: %s" % (col, IQR))
    global outlier_free_list
    global filtered_data

    lower_range = Q1 - 1.5 * IQR
    upper_range = Q3 + 1.5 * IQR
    outlier_free_list = [x for x in data[col] if (
        (x > lower_range) & (x < upper_range))]
    filtered_data = data.loc[data[col].isin(outlier_free_list)] #entsprechende Outlier free rows setzen

counter=0
for i in numeric_columns:
    if counter == 0:
      removeOutliers(df_clean, i)
      counter=counter+1 # einaml mit anfangs df rein, danach mit arbeist variable
    else: 
      removeOutliers(filtered_data, i)
df_clean=filtered_data.copy()

print(df_clean.shape)
df_clean

IQR value for column Displacement ccm is: 735.0
IQR value for column Fuel capacity liters is: 7.0
IQR value for column Power HP is: 68.0
(17972, 8)


,Category,Engine type,"Transmission type,final drive",Displacement ccm,Fuel capacity liters,Power HP,Front brakes,Rear brakes
1,Super motard,"V2, four-stroke",Chain,896.1,12.0,95.2,Double disc,Single disc
2,Sport,"Single cylinder, four-stroke",Chain,124.2,14.5,28.2,Single disc,Single disc
3,Sport,"Single cylinder, four-stroke",Chain,124.2,14.5,15.0,Single disc,Single disc
5,Sport,"Twin, four-stroke",Chain,659.0,15.0,100.0,Double disc,Single disc
6,Sport,"Twin, four-stroke",Chain,659.0,15.0,46.9,Double disc,Single disc
...,...,...,...,...,...,...,...,...
38240,Naked bike,"Two cylinder boxer, four-stroke",Shaft drive (cardan),494.0,14.0,8.6,Expanding brake (drum brake),Expanding brake (drum brake)
38272,Naked bike,"Two cylinder boxer, four-stroke",Shaft drive (cardan),494.0,14.0,8.6,Expanding brake (drum brake),Expanding brake (drum brake)
38305,Naked bike,"Two cylinder boxer, four-stroke",Shaft drive (cardan),494.0,14.0,8.6,Expanding brake (drum brake),Expanding brake (drum brake)
38339,Naked bike,"Two cylinder boxer, four-stroke",Shaft drive (cardan),494.0,14.0,8.6,Expanding brake (drum brake),Expanding brake (drum brake)


In [7]:
text_columns = df_clean.select_dtypes(include=['object', 'string']).columns.tolist()

print(f"Filtere folgende Text-Spalten: {text_columns}\n")

# Für jede Text-Spalte filtern
for col in text_columns:
    print(f"Verarbeite Spalte: {col}")
    
    # Häufigkeiten zählen
    value_counts = df_clean[col].value_counts()
    print(f"  Vorher: {len(value_counts)} unique values, {len(df_clean)} Zeilen")
    
    # Kategorien mit genug Vorkommen identifizieren
    valid_categories = value_counts[value_counts >= min_count].index
    
    # Entfernte Kategorien anzeigen
    removed = value_counts[value_counts < min_count]
    if len(removed) > 0:
        print(f"  Entferne {len(removed)} Kategorien mit < {min_count} Vorkommen:")
        print(f"  {removed.to_dict()}")
    
    # DataFrame filtern
    df_clean = df_clean[df_clean[col].isin(valid_categories)].copy()
    
    print(f"  Nachher: {df_clean[col].nunique()} unique values, {len(df_clean)} Zeilen\n")

#df_clean

Filtere folgende Text-Spalten: ['Category', 'Engine type', 'Transmission type,final drive', 'Front brakes', 'Rear brakes']

Verarbeite Spalte: Category
  Vorher: 13 unique values, 17972 Zeilen
  Nachher: 13 unique values, 17972 Zeilen

Verarbeite Spalte: Engine type
  Vorher: 24 unique values, 17972 Zeilen
  Entferne 15 Kategorien mit < 200 Vorkommen:
  {'In-line six, four-stroke': 93, 'Two cylinder boxer, two-stroke': 71, 'Six cylinder boxer, four-stroke': 63, 'In-line three, two-stroke': 62, 'Square four cylinder': 34, 'Four cylinder boxer, four-stroke': 32, 'V6, four-stroke': 29, 'Diesel': 19, 'In-line four, two-stroke': 12, 'V2, two-stroke': 12, 'Dual disk Wankel': 9, 'Single disk Wankel': 7, 'V4, two-stroke': 6, 'V3, two-stroke': 4, 'Electric': 1}
  Nachher: 9 unique values, 17518 Zeilen

Verarbeite Spalte: Transmission type,final drive
  Vorher: 3 unique values, 17518 Zeilen
  Nachher: 3 unique values, 17518 Zeilen

Verarbeite Spalte: Front brakes
  Vorher: 13 unique values, 1751

In [8]:
# Alex:

# #text_columns.remove("Category")
# categorical_columns = text_columns.copy()

# encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
# encoded = encoder.fit_transform(df_clean[categorical_columns])

# # Als DataFrame mit Namen
# df_encoded = pd.DataFrame(
#     encoded,
#     columns=encoder.get_feature_names_out(categorical_columns),
#     index=df_clean.index
# )


# # Mit Original-DataFrame kombinieren
# df_final = pd.concat([df_clean, df_encoded], axis=1)
# numeric_cols = df_clean.select_dtypes(include=['number']).columns.tolist()
# #numeric_cols.append("Category")
# df_final = pd.concat([df_encoded, df_clean[numeric_cols]], axis=1)

# for col in df_final.columns:
#    df_final[col]=(df_final[col]-df_final[col].mean())/df_final[col].std()


In [10]:
# David 

categorical_columns = text_columns.copy()
categorical_columns.remove("Category")  # ← Category NICHT one-hot-encoden

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded = encoder.fit_transform(df_clean[categorical_columns])

# Als DataFrame mit Namen
df_encoded = pd.DataFrame(
    encoded,
    columns=encoder.get_feature_names_out(categorical_columns),
    index=df_clean.index
)

# Numerische Spalten extrahieren (OHNE Category!)
numeric_cols = df_clean.select_dtypes(include=['number']).columns.tolist()

# NUR numerische Spalten normalisieren
df_numeric_scaled = df_clean[numeric_cols].copy()
for col in numeric_cols:
    df_numeric_scaled[col] = (df_numeric_scaled[col] - df_numeric_scaled[col].mean()) / df_numeric_scaled[col].std()

# Kombinieren: One-Hot (ohne Category) + Numerisch (skaliert) + Category (original)
df_final = pd.concat([df_encoded, df_numeric_scaled, df_clean[['Category']]], axis=1)

In [ ]:
df_final

,"Engine type_In-line four, four-stroke","Engine type_In-line three, four-stroke","Engine type_Single cylinder, four-stroke","Engine type_Single cylinder, two-stroke","Engine type_Twin, four-stroke","Engine type_Twin, two-stroke","Engine type_Two cylinder boxer, four-stroke","Engine type_V2, four-stroke","Engine type_V4, four-stroke","Transmission type,final drive_Belt",...,Front brakes_Expanding brake,Front brakes_Expanding brake (drum brake),Front brakes_Single disc,Rear brakes_Expanding brake,Rear brakes_Expanding brake (drum brake),Rear brakes_Single disc,Displacement ccm,Fuel capacity liters,Power HP,Category
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.765720,-0.436047,1.078033,Super motard
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,-1.008327,0.043085,-0.509866,Sport
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,-1.008327,0.043085,-0.822705,Sport
5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.220796,0.138912,1.191792,Sport
6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.220796,0.138912,-0.066676,Sport
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38240,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,-0.158421,-0.052741,-0.974385,Naked bike
38272,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,-0.158421,-0.052741,-0.974385,Naked bike
38305,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,-0.158421,-0.052741,-0.974385,Naked bike
38339,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,-0.158421,-0.052741,-0.974385,Naked bike


In [11]:
category_mapping = {
    'Custom / cruiser': 'Classic',
    'Enduro / offroad': 'Cross / motocross',
    'ATV': 'Cross / motocross',
    'Allround': 'Naked bike',
    'Sport touring': 'Sport',
    'Super motard': 'Super motard',
    'Touring': 'Touring',
    'Unspecified category': 'Unspecified category',
    'Scooter': 'Scooter',
    # Falls die Hauptkategorien bereits existieren, behalten wir sie
    'Classic': 'Classic',
    'Cross / motocross': 'Cross / motocross',
    'Naked bike': 'Naked bike',
    'Sport': 'Sport'
}
 
# Kategorien umbenennen
df_final['Category'] = df_final['Category'].replace(category_mapping)
print(df_final['Category'].unique())

['Super motard' 'Sport' 'Cross / motocross' 'Scooter' 'Naked bike'
 'Classic' 'Touring' 'Unspecified category']


### abspeichern


In [12]:
#save_path=parent_dir+"/data/joint_data_collection_KategorieVectorized.csv" # Alex
save_path=parent_dir+"/data/joint_data_collection_1801.csv" # David
df_final.to_csv(save_path)